In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [9]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [11]:
print(data.columns)
print(data['label'].value_counts())

Index(['text', 'label'], dtype='object')
label
0    558
1    442
Name: count, dtype: int64


In [21]:
print(data.head())

                                                text  label
0  DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...      1
1                                           Will do.      0
2  Nora--Cheryl has emailed dozens of memos about...      0
3  Dear Sir=2FMadam=2C I know that this proposal ...      1
4                                                fyi      0


### Let's divide the training and test set into two partitions

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)


## Data Preprocessing

In [29]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [46]:
import re

def strip_html(text):
    text = re.sub(r'(?is)<(script|style).*?>.*?(</\1>)', '', str(text))
    text = re.sub(r'(?is)<!--.*?-->', '', text)
    text = re.sub(r'(?s)<[^>]+>', '', text)
    text = re.sub(r'\s+', '', text).strip()
    return text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [47]:
def clean_text(text):
    text = str(text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)      # Keep only letters and spaces
    text = re.sub(r'\b\w\b', ' ', text)           # Remove single characters
    text = re.sub(r'\s+', ' ', text)              # Replace multiple spaces with one
    text = text.strip().lower()                   # Trim and lowercase
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

STOP_WORDS = set(stopwords.words("english"))

def remove_stopwords(text):
    return " ".join(word for word in text.split() if word.lower() not in STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/patrikbobcsok/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [55]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [56]:
def preprocess_series(series: pd.Series) -> pd.Series:
    return (series.astype(str)
            .map(strip_html)
            .map(clean_text)
            .map(remove_stopwords)
            .map(lemmatize_text))

X_train_cleaned = preprocess_series(X_train)
X_test_cleaned  = preprocess_series(X_test)

mask_tr = X_train_cleaned.str.strip().astype(bool)
X_train_cleaned = X_train_cleaned[mask_tr]
y_train = y_train[mask_tr]

mask_te = X_test_cleaned.str.strip().astype(bool)
X_test_cleaned = X_test_cleaned[mask_te]
y_test = y_test[mask_te]

print("Train sizes:", len(X_train_cleaned), len(y_train))

Train sizes: 790 790


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [57]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

ham_texts  = X_train_cleaned[y_train == 0]
spam_texts = X_train_cleaned[y_train == 1]

# Safety again
ham_texts  = ham_texts[ham_texts.str.strip().astype(bool)]
spam_texts = spam_texts[spam_texts.str.strip().astype(bool)]

# Vectorize (letters only)
vec_ham  = CountVectorizer(token_pattern=r'(?u)\b[a-zA-Z]+\b')
vec_spam = CountVectorizer(token_pattern=r'(?u)\b[a-zA-Z]+\b')

ham_bow  = vec_ham.fit_transform(ham_texts)
spam_bow = vec_spam.fit_transform(spam_texts)

ham_top = (pd.DataFrame({'word': vec_ham.get_feature_names_out(),
                         'count': ham_bow.toarray().sum(axis=0)})
           .sort_values('count', ascending=False).head(10))
spam_top = (pd.DataFrame({'word': vec_spam.get_feature_names_out(),
                          'count': spam_bow.toarray().sum(axis=0)})
            .sort_values('count', ascending=False).head(10))

print("Top 10 HAM words:\n", ham_top, "\n")
print("Top 10 SPAM words:\n", spam_top)

Top 10 HAM words:
              word  count
948           fyi     40
3425  unfavorable     27
2701      soffice     27
3488          vol     26
2319      refused     26
3274   thrdenough     25
1194          hvn     24
2809        state     23
1841           na     22
160   amsecretary     21 

Top 10 SPAM words:
           word  count
15415     nbsp    360
4023       com    106
14915       mr     88
25423    yahoo     74
22181        u     73
9848      http     64
24920      www     55
4855   dearsir     49
14620  million     46
2030        aw     45


## Extra features

In [58]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_symbol_list = r"(euro|dollar|pound|€|£|\$)"

suspicious_words  = r"(free|cheap|sex|money|account|bank|fund|transfer|transaction|win|deposit|password)"

money_mark_train = X_train_cleaned.str.contains(money_symbol_list, regex=True).astype(int)
suspicious_train = X_train_cleaned.str.contains(suspicious_words, regex=True).astype(int)
text_len_train   = X_train_cleaned.apply(lambda x: len(x.split()))

money_mark_test = X_test_cleaned.str.contains(money_symbol_list, regex=True).astype(int)
suspicious_test = X_test_cleaned.str.contains(suspicious_words, regex=True).astype(int)
text_len_test   = X_test_cleaned.apply(lambda x: len(x.split()))

print(pd.DataFrame({
    'text': X_train_cleaned.head(),
    'money_mark': money_mark_train.head(),
    'suspicious_words': suspicious_train.head(),
    'text_len': text_len_train.head()
}))

                                                  text  money_mark  \
29   regard mrnelsonsmith kindlyreplymeonmyprivatee...           0   
535  ihavenotbeenabletoreachoscarthisam wearesuppos...           0   
695  humaabedinb mcheckingwithpatonthe kwillworkwit...           0   
557             icanhaveitannouncedhereonmonday ttoday           0   
836  bankofafricaagencesanpedro bp sanpedro coted i...           1   

     suspicious_words  text_len  
29                  0         6  
535                 0         3  
695                 0         5  
557                 0         2  
836                 1        60  


/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/1790663120.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  money_mark_train = X_train_cleaned.str.contains(money_symbol_list, regex=True).astype(int)
/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/1790663120.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  suspicious_train = X_train_cleaned.str.contains(suspicious_words, regex=True).astype(int)
/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/1790663120.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  money_mark_test = X_test_cleaned.str.contains(money_symbol_list, regex=True).astype(int)
/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/1790663120.py

## How would work the Bag of Words with Count Vectorizer concept?

In [59]:
vectorizer = CountVectorizer()

# Fit on training set and transform it into BoW matrix
X_train_bow = vectorizer.fit_transform(X_train_cleaned)

# Convert sparse matrix to DataFrame for readability
bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Show first few rows
print("Vocabulary size:", len(vectorizer.get_feature_names_out()))
print(bow_df.head())

Vocabulary size: 30499
   aa  aaa  aabeiawaeaambiqaceqedeqh  aac  aacute  aacw  aae  \
0   0    0                         0    0       0     0    0   
1   0    0                         0    0       0     0    0   
2   0    0                         0    0       0     0    0   
3   0    0                         0    0       0     0    0   
4   0    0                         0    0       0     0    0   

   aaecaxeebsexbhjbuqdhcrmimoeifekrobhbcsmzuvavynlrchyknoel  aaegmdbsch  aaeh  \
0                                                  0                  0     0   
1                                                  0                  0     0   
2                                                  0                  0     0   
3                                                  0                  0     0   
4                                                  0                  0     0   

   ...  zzwh  zzwqgb  zzwqgdg  zzwqgyw  zzwx  zzxh  zzxmsihdoawxlig  zzz  \
0  ...     0       0        0

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [60]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X_train_cleaned)
print("TF-IDF shape:", X_tfidf.shape)



TF-IDF shape: (790, 30499)


## And the Train a Classifier?

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_cleaned)
X_test_tfidf = tfidf_vectorizer.transform(X_test_cleaned)

# 2. Train classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_tfidf, y_train)

# 3. Predict on test
y_pred = clf.predict(X_test_tfidf)

# 4. Evaluate
from sklearn.metrics import classification_report, accuracy_score
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test Accuracy: 0.9346733668341709
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       124
           1       0.97      0.85      0.91        75

    accuracy                           0.93       199
   macro avg       0.94      0.92      0.93       199
weighted avg       0.94      0.93      0.93       199



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import hstack, csr_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report

# Function to show top words for SPAM and HAM classes
def show_top_words(model, feature_names, top_k=10):
    scores = model.feature_log_prob_[1] - model.feature_log_prob_[0]
    idx_spam = np.argsort(scores)[::-1][:top_k]
    idx_ham  = np.argsort(scores)[:top_k]

    print("\nTop SPAM words:")
    for i in idx_spam:
        print(f"{feature_names[i]:<15}  score={scores[i]:.3f}")

    print("\nTop HAM words:")
    for i in idx_ham:
        print(f"{feature_names[i]:<15}  score={scores[i]:.3f}")

# Train BoW (default params)
bow = CountVectorizer(token_pattern=r"\b[a-zA-Z]{2,}\b")
Xtr_bow = bow.fit_transform(X_train_cleaned)
Xte_bow = bow.transform(X_test_cleaned)

# Train NB (default params)
nb_bow = MultinomialNB()
nb_bow.fit(Xtr_bow, y_train)

# Predict and evaluate
pred_bow = nb_bow.predict(Xte_bow)
print("BoW ONLY — accuracy:", accuracy_score(y_test, pred_bow))
print(classification_report(y_test, pred_bow, target_names=["ham","spam"]))

# Show top words
show_top_words(nb_bow, bow.get_feature_names_out(), top_k=10)

# Train TF-IDF
tfidf = TfidfVectorizer(token_pattern=r"\b[a-zA-Z]{2,}\b")
Xtr_tfidf = tfidf.fit_transform(X_train_cleaned)
Xte_tfidf = tfidf.transform(X_test_cleaned)

# Train NB on TF-IDF
nb_tfidf = MultinomialNB()
nb_tfidf.fit(Xtr_tfidf, y_train)

# Predict and evaluate
pred_tfidf = nb_tfidf.predict(Xte_tfidf)
print("TF-IDF ONLY — accuracy:", accuracy_score(y_test, pred_tfidf))
print(classification_report(y_test, pred_tfidf, target_names=["ham","spam"]))

show_top_words(nb_tfidf, tfidf.get_feature_names_out(), top_k=10)

# Add to the original dataframe two additional indicators (money symbols and suspicious words).
money_pat = r"(euro|dollar|pound|€|£|\$)"
suspi_pat = r"(free|cheap|sex|money|account|bank|fund|transfer|transaction|win|deposit|password)"

# Train flags
money_tr = X_train_cleaned.str.contains(money_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
suspi_tr = X_train_cleaned.str.contains(suspi_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
len_tr   = np.array([len(t.split()) for t in X_train_cleaned]).reshape(-1,1)

# Test flags
money_te = X_test_cleaned.str.contains(money_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
suspi_te = X_test_cleaned.str.contains(suspi_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
len_te   = np.array([len(t.split()) for t in X_test_cleaned]).reshape(-1,1)

# Convert to sparse cols so we can stack with BoW/TF-IDF
def col(x): return csr_matrix(x)
extra_names = ["money_mark","suspicious_words","text_len"]

# Stack BoW with flags
Xtr_bow_extra = hstack([Xtr_bow, col(money_tr), col(suspi_tr), col(len_tr)])
Xte_bow_extra = hstack([Xte_bow, col(money_te), col(suspi_te), col(len_te)])
bow_extra_names = list(bow.get_feature_names_out()) + extra_names

# Train NB on BoW + flags
nb_bow_extra = MultinomialNB()
nb_bow_extra.fit(Xtr_bow_extra, y_train)

# Predict and evaluate
pred_bow_extra = nb_bow_extra.predict(Xte_bow_extra)
print("BoW + FLAGS — accuracy:", accuracy_score(y_test, pred_bow_extra))
print(classification_report(y_test, pred_bow_extra, target_names=["ham","spam"]))

show_top_words(nb_bow_extra, np.array(bow_extra_names), top_k=10)

# Stack TF-IDF with flags
Xtr_tfidf_extra = hstack([Xtr_tfidf, col(money_tr), col(suspi_tr), col(len_tr)])
Xte_tfidf_extra = hstack([Xte_tfidf, col(money_te), col(suspi_te), col(len_te)])
tfidf_extra_names = list(tfidf.get_feature_names_out()) + extra_names

# Train NB on TF-IDF + flags
nb_tfidf_extra = MultinomialNB()
nb_tfidf_extra.fit(Xtr_tfidf_extra, y_train)

# Predict and evaluate
pred_tfidf_extra = nb_tfidf_extra.predict(Xte_tfidf_extra)
print("TF-IDF + FLAGS — accuracy:", accuracy_score(y_test, pred_tfidf_extra))
print(classification_report(y_test, pred_tfidf_extra, target_names=["ham","spam"]))

show_top_words(nb_tfidf_extra, np.array(tfidf_extra_names), top_k=10)

BoW ONLY — accuracy: 0.9698492462311558
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       124
        spam       0.96      0.96      0.96        75

    accuracy                           0.97       199
   macro avg       0.97      0.97      0.97       199
weighted avg       0.97      0.97      0.97       199


Top SPAM words:
nbsp             score=5.229
yahoo            score=3.658
dearsir          score=3.253
million          score=3.191
aw               score=3.169
ig               score=3.004
af               score=2.978
bestregards      score=2.978
iammr            score=2.896
usd              score=2.896

Top HAM words:
fyi              score=-4.373
soffice          score=-3.992
unfavorable      score=-3.992
refused          score=-3.955
vol              score=-3.955
thrdenough       score=-3.918
hvn              score=-3.878
amsecretary      score=-3.751
pmsecretary      score=-3.704
nov              score=-3.368
TF-IDF ONL

/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/4069116629.py:55: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  money_tr = X_train_cleaned.str.contains(money_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/4069116629.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  suspi_tr = X_train_cleaned.str.contains(suspi_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
/var/folders/xx/q0js_gcd29lfhsshk693bdyw0000gn/T/ipykernel_53329/4069116629.py:60: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  money_te = X_test_cleaned.str.contains(money_pat, flags=re.I, regex=True).astype(int).values.reshape(-1,1)
/var/folders/xx/q0js_gc